In [1]:
import sklearn
import numpy as np
import pandas as pd

import torch
cuda_available = torch.cuda.is_available()

from simpletransformers.ner import NERModel, NERArgs

# from ner_eval import collect_named_entities
# from ner_eval import compute_metrics
# from ner_eval import compute_precision_recall_wrapper

# from sklearn_crfsuite import metric
from nervaluate import Evaluator

In [5]:
model_name='indolem/indobert-base-uncased'

In [6]:
train_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\train.csv")
eval_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\val.csv")
test_df =pd.read_csv("..\..\..\Dataset\\simpletransformers\\test.csv")

In [7]:
label_list = [
    "O", 
    "B-PER", "I-PER",
    "B-ORG", "I-ORG",
    "B-LOC", "I-LOC",
    "B-PROD", "I-PROD",
    "B-WA", "I-WA",
    "B-EV", "I-EV",
]

In [8]:
# Create a NERModel
model_args=NERArgs()
# model_args.train_batch_size=64
# model_args.overwrite_output_dir=True
model_args = {
    "output_dir": "outputs/",
    "cache_dir": "cache_dir/",

    "fp16": True,
    "fp16_opt_level": "O1",
    "max_seq_length": 128,
    "train_batch_size": 8,
    "gradient_accumulation_steps": 1,
    "eval_batch_size": 8,
    # "num_train_epochs": 1,
    "num_train_epochs": 5,
    "weight_decay": 0,
    "learning_rate": 4e-5,

    "adam_epsilon": 1e-8,
    "warmup_ratio": 0.06,
    "warmup_steps": 0,
    "max_grad_norm": 1.0,

    "logging_steps": 50,
    "save_steps": 2000,

    "overwrite_output_dir": True,
    "reprocess_input_data": False,
    "evaluate_during_training": True,

    "do_lower_case": False, #True if using uncased models
}


In [9]:
# model = NERModel('bert', 
#                 'outputs/checkpoint-594-epoch-1',
#                 args=model_args,
#                 labels=label_list,
#                 use_cuda=cuda_available,
#                 )
# # result, model_outputs, wrong_preds = model.eval_model(eval_df)
# # result

In [10]:
model = NERModel('bert',
                model_name,
                args=model_args,
                labels=label_list,
                use_cuda=cuda_available,
                )

Downloading: 100%|██████████| 0.99k/0.99k [00:00<00:00, 1.01MB/s]
Downloading: 100%|██████████| 424M/424M [00:21<00:00, 20.8MB/s] 
Some weights of the model checkpoint at indolem/indobert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassificati

In [11]:
model.train_model(train_df,  eval_data=eval_df)

2021-12-11 19:30:21 simpletransformers.ner.ner_model INFO:  Converting to features started.
Epoch 1 of 5:   0%|          | 0/5 [00:00<?, ?it/s]d:\ner-idn-tweet\env\lib\site-packages\torch\optim\lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
Epochs 0/5. Running Loss:    0.2091: 100%|██████████| 594/594 [00:54<00:00, 10.94it/s]
2021-12-11 19:31:25 simpletransformers.ner.ner_model INFO:  Converting to features started.
Epochs 1/5. Running Loss:    0.1728: 100%|██████████| 594/594 [00:52<00:00, 11.34it/s]
2021-12-11 19:32:30 simpletransformers.ner.n

(2970,
 defaultdict(list,
             {'global_step': [594, 1188, 1782, 2000, 2376, 2970],
              'train_loss': [0.2090768665075302,
               0.1728057563304901,
               0.16107790172100067,
               0.3203752636909485,
               0.14825043082237244,
               0.24933025240898132],
              'eval_loss': [0.3622170786729595,
               0.32053411316951647,
               0.31796830922565206,
               0.32607817897360597,
               0.3206847670094279,
               0.3352316556210886],
              'precision': [0.5302013422818792,
               0.541267942583732,
               0.514668039114771,
               0.5498073747936159,
               0.528793189784677,
               0.5242334322453017],
              'recall': [0.27952233524988945,
               0.40026536930561696,
               0.44228217602830605,
               0.44183989385227773,
               0.4670499778858912,
               0.4688191065900044],
       

In [12]:
result, model_outputs, y_pred = model.eval_model(test_df)

2021-12-11 19:35:38 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Evaluation: 100%|██████████| 186/186 [00:03<00:00, 61.40it/s]
2021-12-11 19:35:47 simpletransformers.ner.ner_model INFO: {'eval_loss': 0.2190948549197406, 'precision': 0.6952532907857998, 'recall': 0.6152488528062124, 'f1_score': 0.652808988764045}


In [13]:
result

{'eval_loss': 0.2190948549197406,
 'precision': 0.6952532907857998,
 'recall': 0.6152488528062124,
 'f1_score': 0.652808988764045}

In [14]:
df_x_test=test_df.groupby('sentence_id')['words'].apply(list)
x_test=list(df_x_test)

df_y_test=test_df.groupby('sentence_id')['labels'].apply(list)
y_test=list(df_y_test)

In [15]:
predictions, raw_outputs = model.predict(
    # ["@PolsekAlian PolriBerikan inovasi-inovasi Polri yang memudahkan masyarakat, kegiatan pemolisian masyarakat dan seluruh tindakan Kepolisian dalam menghadapi gangguan Kamtibmas, dan lain-lain - Jenderal Polisi Drs. Listyo Sigit Prabowo, https://t.co/f8GZ6VPqPD. - Kapolri #PolriTVRadioPresisi"],
    ["@ShopeeID Bismillah yok menang ���� SHOPEE @ShopeeID #44ShopeeMamamoo #ShopeexMAMAMOO #AdaMamamoodiShopee"], 
    ["@OPPOIndonesia ✍Butuh memori besar 128GB di OPPO A54 buat simpan file-file tugas kuliah dan testimoni online shop -ku ��Semoga oppo A54 bisa menunjang penghasilan keluargaku �� @OPPOIndonesia @nisa_rkt @Bebheey @kus_ica @Erna19_ @Saftriyuni #OPPOA54 https://t.co/bMYIiRb4QF �� 2.921"]
        )

2021-12-11 19:35:47 simpletransformers.ner.ner_model INFO:  Converting to features started.
Running Prediction: 100%|██████████| 1/1 [00:00<00:00, 66.65it/s]


In [16]:
predictions

[[{'@ShopeeID': 'O'},
  {'Bismillah': 'O'},
  {'yok': 'O'},
  {'menang': 'O'},
  {'����': 'O'},
  {'SHOPEE': 'O'},
  {'@ShopeeID': 'O'},
  {'#44ShopeeMamamoo': 'O'},
  {'#ShopeexMAMAMOO': 'O'},
  {'#AdaMamamoodiShopee': 'O'}]]

In [57]:
#Now we create the model again using the best estimators

print(sklearn.metrics.classification_report(
    y_test, y_pred, labels=label_list, digits=3
    ))

ValueError: You appear to be using a legacy multi-label data representation. Sequence of sequences are no longer supported; use a binary array or sparse matrix instead - the MultiLabelBinarizer transformer can convert to this format.

In [17]:
evaluator = Evaluator(y_test, y_pred, tags=['LOC', 'PER', 'ORG', 'PROD', 'WA', 'EV'], loader="list")

In [18]:
results, results_per_tag = evaluator.evaluate()

In [19]:
results['partial']

{'correct': 1869,
 'incorrect': 0,
 'partial': 272,
 'missed': 716,
 'spurious': 366,
 'possible': 2857,
 'actual': 2507,
 'precision': 0.7997606701236538,
 'recall': 0.7017850892544627,
 'f1': 0.7475764354958986}

In [20]:
results['exact']

{'correct': 1869,
 'incorrect': 272,
 'partial': 0,
 'missed': 716,
 'spurious': 366,
 'possible': 2857,
 'actual': 2507,
 'precision': 0.7455125648185081,
 'recall': 0.6541827091354567,
 'f1': 0.6968680089485458}

In [21]:
with open("./results_1_batch.txt", "w") as f:
    str_results = repr(results)
    f.write(str_results)

In [22]:
with open("./results_per_tag_1_batch.txt", "w") as f:
    str_results_per_tag = repr(results_per_tag)
    f.write(str_results_per_tag)

In [ ]:
import re
for i in range(len(y_test)):
    if len(y_test[i]) != len(y_pred[i]):
        print(x_test[i])
        print(len(x_test[i]))
        print("y_test       ",len(y_test[i]),y_test[i])
        print("y_pred before",len(y_pred[i]),y_pred[i])
        for j in range(len(x_test[i])):
            # if '�' in x_test[i][j]: 
            qmark=re.match(r'^�+$',x_test[i][j])
            if qmark:
                print(qmark)
                y_pred[i].insert(j,'O')
        print("y_pred after ",len(y_pred[i]), y_pred[i])
        print()

['@Yanua02', '@BatuKepla', '@Karsono12732862', '@Andreas50772732', '@AndreMa32495641', '@victorpangkey', '@andriskhas', '@AbyManyuu', '@lizaariani5', '@Nana_Rosaa', '_', '@dwi2dh25', '@BaswirOb', '@Bang__Mus', '@745L023N6', '@Rijani54531379', '@Dewi52502425', '@Celes_being1074', '@Yatimpi97149270', '@begoeloh', '@FEMYLIA_K', '@FKARIN3', '@PRINCESLAMPIR', '@aewin86', '@Elina_Vay', '@amandagempita', '@CynthiaEllenna', '@dchyntiabella', '@Permata_Dewi06', '@Dewianggra91', '@Dewi37492961', '@Putri_Sulita', '@tasya_aa77', '@bunga_lao', '@Rismaliez85', '@SonyaAmriani', '@gituLok', '@hennyindah1404', '@WiwidNurwidaya1', '@OniMeniq74', '@maita_omerta', '@Ek4_Puspita', '@Mahrus30127858', '@Zoer_Riya', '@Reny_soepomo', '@BersamaSahabat4', '@jokowi', '@KPK_RI', '@PerekonomianRI', '@airlangga_hrt', '@KemenkeuRI', 'Infrastruktur', 'adalah', 'fondasi', 'bagi', 'rumah', 'indah', 'Indonesia', 'yg', 'aka', 'dibangun', 'di', 'era', 'pasca', 'Jokowi', 'Jokowi', 'yg', 'bangun', 'fondasi', 'rumahnya', ',',

NameError: name 're' is not defined